## Data Processing

In [2]:
import numpy as np
import pandas as pd
import random as rand

In [3]:
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(2018)
import nltk
nltk.download('wordnet')
from nltk.stem.porter import *
stemmer = PorterStemmer()

/Users/siqifu/anaconda3/lib/python3.7/site-packages/smart_open/ssh.py:34: UserWarning: paramiko missing, opening SSH/SCP/SFTP paths will be disabled.  `pip install paramiko` to suppress
  warnings.warn('paramiko missing, opening SSH/SCP/SFTP paths will be disabled.  `pip install paramiko` to suppress')
[nltk_data] Downloading package wordnet to /Users/siqifu/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [4]:
df = pd.read_csv('articles2.csv').head(1000)

In [5]:
df_text = df[['content']]

In [7]:
df_text['index'] = df_text.index

/Users/siqifu/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [8]:
documents = df_text

In [9]:
def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))
def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
    return result

In [10]:
processed_docs = documents['content'].map(preprocess)
processed_docs[:5]

0    [patriot, peter, berg, thriller, recreat, bost...
1    [nors, mytholog, human, world, creat, pantheon...
2    [democraci, work, increasingli, divers, nation...
3    [updat, januari, press, confer, juli, donald, ...
4    [updat, month, equivoc, origin, cyberattack, t...
Name: content, dtype: object

In [11]:
dictionary = gensim.corpora.Dictionary(processed_docs)
count = 0
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 10:
        break

0 accur
1 action
2 aftermath
3 agent
4 agon
5 alexand
6 ambigu
7 analysi
8 angri
9 ask
10 attack


In [12]:
def lemmatize_stemming(text):
    '''
    Lenmmatize and stem the text.
    '''
    return PorterStemmer().stem(WordNetLemmatizer().lemmatize(text, pos='v'))

def preprocess(text):
    '''
    Preprocess the text.
    '''
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
    return result

processed_docs = documents['content'].map(preprocess)

vocabulary = gensim.corpora.Dictionary(processed_docs)
#vocabulary.filter_extremes(no_below=5, no_above=0.1, keep_n=100)
bow_corpus = [vocabulary.doc2bow(doc) for doc in processed_docs]

doc = [dict(bow) for bow in bow_corpus]

## LDA Algorithm

In [13]:
import numba
import numpy as np
from scipy.special import digamma, polygamma
from numba import jit, float64, int64

In [16]:
class LDA_test1:
    
    @staticmethod
    def _convergence_(new, old, epsilon = 1.0e-3):
        '''
        Check convergence.
        '''
        return np.all(np.abs(new - old)) < epsilon
    
    
    @staticmethod
    def _normalization(x):
        '''
        Normalize the input.
        '''
        return x/np.sum(x)
    
  
    @staticmethod
    def _normalization_row(x):
        return x/np.sum(x,1)[:,None]
    

    @staticmethod
    def _normalization_col(x):
        '''
        Normalize a matrix. 
        Each element is divided by the corresponding column sum.
        '''
        return x/np.sum(x,0)
    
    
    @staticmethod
    @numba.jit(cache = True)
    def _accumulate_Phi(beta, Phi, doc):
        '''
        This function accumulates the effect of Phi_new from all documents after e step.
        beta is V*k matrix.
        Phi is N_d * k matrix.
        Return updated beta.
        '''
        
        beta[list(doc.keys()),:] += np.diag(list(doc.values())) @ Phi
    
        return beta
    
    
    def __init__(self, k, max_em_iter=10, max_alpha_iter=10, max_Estep_iter=10):
        self._k = k
        self._max_em_iter = max_em_iter
        self._max_alpha_iter = max_alpha_iter
        self._max_Estep_iter = max_Estep_iter
        
     
     
    def initializaiton(self, V):
        '''
        Initialize alpha and beta. 
        alpha is a k-dim vector. beta is V*k matrix.
        '''
        
        k = self._k
        np.random.seed(12345)
        alpha = self._normalization(np.random.uniform(size = k))
    
        beta = np.random.dirichlet(alpha, V)
        
        return alpha, beta
    
    
    def Estep(self, doc, alpha, beta, N_d):
        '''
        E step for a document, which calculate the posterior parameters.
        beta_old and alpha-old is coming from previous iteration.
        Return Phi and gamma  of a document.
        '''
        
        k = self._k
        max_iter = self._max_Estep_iter
        
        gamma_old = alpha + np.ones(k) * N_d/k
        row_index = list(doc.keys())
        word_count = np.array(list(doc.values()))
    
        for i in range(max_iter):
            # Update Phi
            Phi_exp = np.exp(digamma(gamma_old))
            Phi = beta[row_index,:] @ np.diag(Phi_exp)
            Phi_new = self._normalization_row(Phi)
        
            # Update gamma
            Phi_sum = Phi_new.T @ word_count[:,None] # k-dim
            gamma_new = alpha + Phi_sum.T[0]
        
            # Converge or not
            if (i>0) & self._convergence_(gamma_new, gamma_old):
                break
            else:
                gamma_old = gamma_new.copy()
    
            
        return gamma_new, Phi_new
    

    def newton_raphson(self, alpha_old, gamma_matrix):
        '''
        This function uses New Raphson method to update alpha in the M step.
        alpha_old is a k-dim vector.
        gamma_matrix is a M * k matrix which stores all gamma from M documents.
        Return updated alpha.
        '''

        k = self._k
        max_iter = self._max_alpha_iter
        
        M = gamma_matrix.shape[0]
        pg = np.sum(digamma(gamma_matrix), 0) - np.sum(digamma(np.sum(gamma_matrix, 1)))
        alpha_new = alpha_old.copy()
    
        for t in range(max_iter):
        
            alpha_sum = np.sum(alpha_old)
            g = M * (digamma(alpha_sum) - digamma(alpha_old)) + pg
            h = -M * polygamma(1, alpha_old)
            z = M * polygamma(1, alpha_sum)
            c = np.sum(g/h)/(z**(-1.0) + np.sum(h**(-1.0)))
        
            delta = (g-c)/h
            alpha_new -= delta
        
            if np.any(alpha_new) < 0:
                alpha_new = self.newton_raphson(alpha_old/10, gamma_matrix)
                return alpha_new
        
            if (t > 1) & self._convergence_(delta, np.zeros((1,k))):
                break
            else:
                alpha_old = alpha_new.copy()
            
        return alpha_new
    
    
    @numba.jit(cache = True)
    def E(self, doc, alpha_old, beta_old, beta_new, gamma_matrix, N_d, M):
        for i in range(M):
            gamma, Phi = self.Estep(doc[i], alpha_old, beta_old, N_d[i])
            beta_new = self._accumulate_Phi(beta_new, Phi, doc[i])
            gamma_matrix[i,:] = gamma
        return beta_new, gamma_matrix
    

    def fit(self, doc, vocabulary):
        '''
        Latent Dirichlet Allocation Model.
        doc is a set of documents, each document is a dictionary.
        vocabulary contains the words in all documents.
        Return updated alpha and beta.
        '''
            
        k = self._k
        max_iter = self._max_em_iter
            
        N_d = [len(d) for d in doc] # Get the length of each document.
        V = len(vocabulary) # Get the length of vocabulary
        M = len(doc) # Get the document number.
    
       # Initialize alpha, beta and the statistics od gamma
        alpha_new, beta_new = self.initializaiton(V)
        gamma_matrix = np.zeros((M, k))
    
        for iter in range(max_iter):
            beta_old = beta_new.copy()
            alpha_old = alpha_new.copy()
            
            # E step
            beta_new, gamma_matrix = self.E(doc, alpha_old, beta_old, beta_new, gamma_matrix, N_d, M)
        
            # M step
            alpha_new = self.newton_raphson(alpha_old, gamma_matrix)
            beta_new = self._normalization_col(beta_new)
        
            # check convergence
            if self._convergence_(alpha_new, alpha_old) & self._convergence_(np.sum(beta_new,0), np.sum(beta_old,0)):
                break
        
        return alpha_new, beta_new


## Fit model to real data

In [52]:
lda_model1 = LDA_test1(k=10) 
#%timeit alpha, beta = lda_model1.fit(doc, vocabulary)

In [53]:
alpha, beta = lda_model1.fit(doc, vocabulary)

In [54]:
def get_topic(alpha_list, topic_num):
    return list(map(alpha_list.index, heapq.nlargest(topic_num, alpha_list)))

In [55]:
def get_words(words_list, word_num):
    '''
    Get the words with largetest probilities in a specific topic.
    words_list is a list of probability of words under a specific topic.
    word_num is the number of words that we return.
    Return the index of words in vocabulary.
    '''
    return list(map(words_list.index, heapq.nlargest(word_num, words_list)))

In [56]:
import heapq

In [57]:
alpha_list = list(alpha)
get_topic(alpha_list, 10)

[6, 0, 8, 5, 7, 9, 4, 1, 3, 2]

In [93]:
words_top = get_words(list(beta[:,4]), 10)

In [94]:
def word_to_list(index):
    """transform the top_words into a list"""
    topic = []
    words_top = get_words(list(beta[:,index]), 10)
    for i in words_top:
        topic.append(vocabulary[i])
    return topic

In [95]:
list1 = word_to_list(4)
list2 = word_to_list(3)
list3 = word_to_list(2)

In [96]:
dict = {'President Election': list1, 'Medical': list2, 'Astronomy': list3}  
result = pd.DataFrame(dict) 
result 

,President Election,Medical,Astronomy
0,trump,patient,space
1,republican,doctor,game
2,report,studi,earth
3,elect,pain,speci
4,campaign,perform,planet
5,democrat,team,episod
6,hous,vaccin,bail
7,say,show,mar
8,accord,pruitt,nasa
9,make,life,moon
